In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys
sys.path.append('/kaggle/input/training-and-ensemble')
sys.path.append('/kaggle/input/zip-file')

In [3]:
import numpy as np
import pandas as pd
import torch
# Import các thư viện cần thiết
from sklearn.model_selection import train_test_split
# Import các lớp tùy chỉnh của bạn
from SVCSpecific import create_specific_svc_pipeline 
from SingleModel import SingleModel
from EnsembleModel import EnsembleModel
from AudioDataset import AudioDataset, audio_collate_fn
from AudioFeatureExtractor import BaseFeatureExtractor, WavLMFeatureExtractor, HubertFeatureExtractor, Wav2Vec2EncoderFeatureExtractor
from audio_evaluation import AudioMisclassificationAnalyzer
from MisclassificationReporter import MisclassificationReporter

# Import các lớp/hàm từ thư viện Hugging Face
from transformers import AutoFeatureExtractor, AutoModel
# Import joblib để lưu trữ hay nạp mô hình từ .pkl
import joblib

2025-05-10 02:37:16.059224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746844636.239659      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746844636.293332      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Đường dẫn gốc đến dữ liệu audio đã được phân loại vào các thư mục con
AUDIO_BASE_DIR = "/kaggle/input/data-speechclassification-labeling/DatasetLabeling/Audio_byClass" # THAY THẾ bằng đường dẫn THỰC TẾ của bạn

# Danh sách các tên lớp (tương ứng với tên thư mục con trong AUDIO_BASE_DIR)
CLASS_NAMES = ["Anger", "Happiness", "Sadness", "Neutral state"] # THAY THẾ bằng danh sách lớp THỰC TẾ của bạn

HF_MODEL_NAME_Wavlm = "microsoft/wavlm-large" 
HF_MODEL_NAME_Wav2vec = "facebook/wav2vec2-large-robust" 
HF_MODEL_NAME_Hubert = "facebook/hubert-large-ll60k" 
HF_MODEL_NAME_unispeech = "microsoft/unispeech-sat-large-sv" 
HF_MODEL_NAME_nest ="nvidia/ssl_en_nest_large_v1.0"
HF_MODEL_NAME_canary = "nvidia/canary-1b-flash" 

# Đường dẫn để lưu/nạp danh sách
AUDIO_LIST_CSV_PATH = "audio_files_list.csv"
# EXTRACTED_FEATURES_CSV_PATH = f"features_{HF_MODEL_NAME_Wavlm.replace('/', '_')}.csv"

# Đường dẫn file CSV cho báo cáo misclassification tổng hợp
REPORT_BASE_DIR = "summary_report.csv"

BATCH_SIZE = 16

ensemble_predictions_dict = {} # Dictionary để lưu dự đoán từ các mô hình/chiến lược cho báo cáo so sánh
list_of_aud_hub_wav = []
list_of_aud_hub = []
list_of_hub_wav = []

In [5]:
print("--- Bắt đầu quy trình ---")

print("\nBước 1: Chuẩn bị danh sách file audio (toàn bộ dataset).")
all_audio_files_labels = []
all_audio_files_labels = AudioDataset.collect_audio_files(AUDIO_BASE_DIR, CLASS_NAMES)
if all_audio_files_labels:
    temp_dataset = AudioDataset(audio_files_list=all_audio_files_labels)
    temp_dataset.save_to_csv(AUDIO_LIST_CSV_PATH)
    del temp_dataset
print(f"Tổng số file audio thu thập được: {len(all_audio_files_labels)}")
# Tạo AudioDataset cho TOÀN BỘ dataset
full_dataset = AudioDataset(audio_files_list=all_audio_files_labels)
print(f"Full dataset có {len(full_dataset)} file.")

--- Bắt đầu quy trình ---

Bước 1: Chuẩn bị danh sách file audio (toàn bộ dataset).
Đang thu thập file audio từ thư mục: /kaggle/input/data-speechclassification-labeling/DatasetLabeling/Audio_byClass
Tìm thấy tổng cộng 5184 file audio hợp lệ.
AudioDataset được khởi tạo với 5184 file.
Đang lưu danh sách file audio vào audio_files_list.csv...
Danh sách file audio đã được lưu thành công.
Tổng số file audio thu thập được: 5184
AudioDataset được khởi tạo với 5184 file.
Full dataset có 5184 file.


### Wavlm

In [6]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Wavlm.csv" # Sửa tên file để phân biệt

extractor = WavLMFeatureExtractor(HF_MODEL_NAME_Wavlm)
# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    BaseFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Wavlm = np.array([item[0] for item in extracted_full_data])
    labels_list_Wavlm = [item[1] for item in extracted_full_data]
    filepaths_list_Wavlm = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del extractor
    
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")



Bước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.
Đang sử dụng thiết bị: cuda
Khởi tạo BaseFeatureExtractor cho mô hình: microsoft/wavlm-large
Đang nạp Processor: microsoft/wavlm-large


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Đang nạp Mô hình WavLM: microsoft/wavlm-large


config.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Cảnh báo: Không thể kiểm tra cấu trúc đầu ra của mô hình WavLM bằng dummy input: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 1, 16000]
WavLMFeatureExtractor đã được khởi tạo thành công.

Bắt đầu trích xuất đặc trưng (WavLM) với batch size 16...


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


  Đã xử lý 5184/5184 mẫu...
Hoàn thành duyệt các batch (WavLM).
Tổng số mẫu đã trích xuất đặc trưng thành công (WavLM): 5184

Hoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Wavlm.csv.
Đang lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Wavlm.csv...
Đặc trưng và metadata đã được lưu thành công.

--- Kết thúc quy trình trích xuất đặc trưng ---


In [7]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Wavlm.shape}, Nhãn: {len(labels_list_Wavlm)}, Filepaths: {len(filepaths_list_Wavlm)}")
assert features_np_Wavlm.shape[0] == len(labels_list_Wavlm) == len(filepaths_list_Wavlm), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Wavlm)}")


--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---
Shape đặc trưng: (5184, 1024), Nhãn: 5184, Filepaths: 5184
Các lớp tìm thấy trong nhãn: ['Anger' 'Happiness' 'Neutral state' 'Sadness']


In [8]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Wavlm, labels_list_Wavlm, filepaths_list_Wavlm, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")


Bước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.
Đã chia dữ liệu:
  Train: X_train shape (4147, 1024), y_train len 4147, train_filepaths len 4147
  Test: X_test shape (1037, 1024), y_test len 1037, test_filepaths len 1037


In [9]:
SVC_PARAMS = {'class_weight': None, 'probability': False}
svc_pipeline = create_specific_svc_pipeline(**SVC_PARAMS)
# Tạo đối tượng SingleModel cho SVC
model_Wavlm_b_svc = SingleModel(
    name="Wavlm SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wavlm_b_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wavlm_b_svc.evaluate()
if model_Wavlm_b_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wavlm_b_svc.get_name()] = model_Wavlm_b_svc.get_predictions()

Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': False, 'class_weight': None}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Wavlm SVM

--- Bắt đầu pipeline cho mô hình: Wavlm SVM ---
Đang huấn luyện mô hình: Wavlm SVM...
Hoàn thành huấn luyện mô hình: Wavlm SVM.
Đang dự đoán với mô hình: Wavlm SVM...
Hoàn thành dự đoán với mô hình: Wavlm SVM.
Bỏ qua lấy xác suất và tính độ bất định cho 'Wavlm SVM' vì mô hình không có predict_proba.
--- Hoàn thành pipeline cho mô hình: Wavlm SVM ---

--- Báo cáo Đánh giá cho Mô hình: Wavlm SVM ---
               precision    recall  f1-score   support

        Anger       0.78      0.76      0.77       366
    Happiness       0.69      0.18      0.29       122
      Sadness       0.61      0.77      0.68       348
Neutral state       0.69      0.73      0.71       201

     accuracy                           0.69      1037
    macro avg       0.69      0.61      0.61      103

In [10]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Wavlm_svc = SingleModel(
    name="Wavlm SVM For Ensemble",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wavlm_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wavlm_svc.evaluate()
list_of_aud_hub_wav.append(model_Wavlm_svc)
list_of_hub_wav.append(model_Wavlm_svc)
if model_Wavlm_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wavlm_svc.get_name()] = model_Wavlm_svc.get_predictions()


Bước 4:## Chuẩn bị Mô hình SVC ##
Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': True, 'class_weight': 'balanced'}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Wavlm SVM For Ensemble

--- Bắt đầu pipeline cho mô hình: Wavlm SVM For Ensemble ---
Đang huấn luyện mô hình: Wavlm SVM For Ensemble...
Hoàn thành huấn luyện mô hình: Wavlm SVM For Ensemble.
Đang dự đoán với mô hình: Wavlm SVM For Ensemble...
Hoàn thành dự đoán với mô hình: Wavlm SVM For Ensemble.
Đang lấy xác suất dự đoán với mô hình: Wavlm SVM For Ensemble...
Hoàn thành lấy xác suất dự đoán với mô hình: Wavlm SVM For Ensemble.
Đã tính toán độ bất định cho mô hình: Wavlm SVM For Ensemble.
--- Hoàn thành pipeline cho mô hình: Wavlm SVM For Ensemble ---

--- Báo cáo Đánh giá cho Mô hình: Wavlm SVM For Ensemble ---
               precision    recall  f1-score   support

        Anger       0.79      0.76      0.78       366
    Happiness       0.54 

In [11]:
joblib.dump(model_Wavlm_svc,"model_Wavlm_svc.pkl")

['model_Wavlm_svc.pkl']

### Hubert

In [12]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Hubert.csv" # Sửa tên file để phân biệt

extractor = HubertFeatureExtractor(HF_MODEL_NAME_Hubert)

# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    BaseFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Hubert = np.array([item[0] for item in extracted_full_data])
    labels_list_Hubert = [item[1] for item in extracted_full_data]
    filepaths_list_Hubert = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del extractor
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")



Bước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.
Đang sử dụng thiết bị: cuda
Khởi tạo BaseFeatureExtractor cho mô hình: facebook/hubert-large-ll60k
Đang nạp Processor: facebook/hubert-large-ll60k


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Đang nạp Mô hình HuBERT: facebook/hubert-large-ll60k


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Cảnh báo: Không thể kiểm tra cấu trúc đầu ra của mô hình HuBERT bằng dummy input: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 1, 16000]
HubertFeatureExtractor đã được khởi tạo thành công.

Bắt đầu trích xuất đặc trưng (HuBERT) với batch size 16...
  Đã xử lý 5184/5184 mẫu...
Hoàn thành duyệt các batch (HuBERT).
Tổng số mẫu đã trích xuất đặc trưng thành công (HuBERT): 5184

Hoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Hubert.csv.
Đang lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Hubert.csv...
Đặc trưng và metadata đã được lưu thành công.

--- Kết thúc quy trình trích xuất đặc trưng ---


In [13]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Hubert.shape}, Nhãn: {len(labels_list_Hubert)}, Filepaths: {len(filepaths_list_Hubert)}")
assert features_np_Hubert.shape[0] == len(labels_list_Hubert) == len(filepaths_list_Hubert), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Hubert)}")


--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---
Shape đặc trưng: (5184, 1024), Nhãn: 5184, Filepaths: 5184
Các lớp tìm thấy trong nhãn: ['Anger' 'Happiness' 'Neutral state' 'Sadness']


In [14]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Hubert, labels_list_Hubert, filepaths_list_Hubert, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")


Bước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.
Đã chia dữ liệu:
  Train: X_train shape (4147, 1024), y_train len 4147, train_filepaths len 4147
  Test: X_test shape (1037, 1024), y_test len 1037, test_filepaths len 1037


In [15]:
svc_pipeline = create_specific_svc_pipeline(**SVC_PARAMS)
# Tạo đối tượng SingleModel cho SVC
model_Hubert_b_svc = SingleModel(
    name="Hubert SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Hubert_b_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Hubert_b_svc.evaluate()
if model_Hubert_b_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Hubert_b_svc.get_name()] = model_Hubert_b_svc.get_predictions()

Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': False, 'class_weight': None}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Hubert SVM

--- Bắt đầu pipeline cho mô hình: Hubert SVM ---
Đang huấn luyện mô hình: Hubert SVM...
Hoàn thành huấn luyện mô hình: Hubert SVM.
Đang dự đoán với mô hình: Hubert SVM...
Hoàn thành dự đoán với mô hình: Hubert SVM.
Bỏ qua lấy xác suất và tính độ bất định cho 'Hubert SVM' vì mô hình không có predict_proba.
--- Hoàn thành pipeline cho mô hình: Hubert SVM ---

--- Báo cáo Đánh giá cho Mô hình: Hubert SVM ---
               precision    recall  f1-score   support

        Anger       0.70      0.76      0.73       366
    Happiness       0.67      0.08      0.15       122
      Sadness       0.60      0.71      0.65       348
Neutral state       0.68      0.74      0.71       201

     accuracy                           0.66      1037
    macro avg       0.66      0.57      0.56

In [16]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Hubert_svc = SingleModel(
    name="Hubert SVM For Ensemble",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Hubert_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Hubert_svc.evaluate()
list_of_aud_hub_wav.append(model_Hubert_svc)
list_of_hub_wav.append(model_Hubert_svc)
list_of_aud_hub.append(model_Hubert_svc)
if model_Hubert_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Hubert_svc.get_name()] = model_Hubert_svc.get_predictions()


Bước 4:## Chuẩn bị Mô hình SVC ##
Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': True, 'class_weight': 'balanced'}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Hubert SVM For Ensemble

--- Bắt đầu pipeline cho mô hình: Hubert SVM For Ensemble ---
Đang huấn luyện mô hình: Hubert SVM For Ensemble...
Hoàn thành huấn luyện mô hình: Hubert SVM For Ensemble.
Đang dự đoán với mô hình: Hubert SVM For Ensemble...
Hoàn thành dự đoán với mô hình: Hubert SVM For Ensemble.
Đang lấy xác suất dự đoán với mô hình: Hubert SVM For Ensemble...
Hoàn thành lấy xác suất dự đoán với mô hình: Hubert SVM For Ensemble.
Đã tính toán độ bất định cho mô hình: Hubert SVM For Ensemble.
--- Hoàn thành pipeline cho mô hình: Hubert SVM For Ensemble ---

--- Báo cáo Đánh giá cho Mô hình: Hubert SVM For Ensemble ---
               precision    recall  f1-score   support

        Anger       0.73      0.73      0.73       366
    Happiness 

In [17]:
joblib.dump(model_Hubert_svc,"model_Hubert_svc.pkl")

['model_Hubert_svc.pkl']

### Wav2vec

In [18]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Wav2vec.csv" # Sửa tên file để phân biệt

extractor = Wav2Vec2EncoderFeatureExtractor(HF_MODEL_NAME_Wav2vec)

# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    BaseFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Wav2vec = np.array([item[0] for item in extracted_full_data])
    labels_list_Wav2vec = [item[1] for item in extracted_full_data]
    filepaths_list_Wav2vec = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del extractor
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


Bước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.
Đang sử dụng thiết bị: cuda
Khởi tạo BaseFeatureExtractor cho mô hình: facebook/wav2vec2-large-robust
Đang nạp Processor: facebook/wav2vec2-large-robust


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Đang nạp Mô hình Wav2Vec2 (Encoder): facebook/wav2vec2-large-robust


pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Cảnh báo: Không thể kiểm tra cấu trúc đầu ra của mô hình Wav2Vec2 bằng dummy input: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 1, 16000]
Wav2Vec2EncoderFeatureExtractor đã được khởi tạo thành công.

Bắt đầu trích xuất đặc trưng (Wav2Vec2) với batch size 16...


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

  Đã xử lý 5184/5184 mẫu...
Hoàn thành duyệt các batch (Wav2Vec2).
Tổng số mẫu đã trích xuất đặc trưng thành công (Wav2Vec2): 5184

Hoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Wav2vec.csv.
Đang lưu đặc trưng và metadata vào features_with_path_HF_MODEL_NAME_Wav2vec.csv...
Đặc trưng và metadata đã được lưu thành công.

--- Kết thúc quy trình trích xuất đặc trưng ---


In [19]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Wav2vec.shape}, Nhãn: {len(labels_list_Wav2vec)}, Filepaths: {len(filepaths_list_Wav2vec)}")
assert features_np_Wav2vec.shape[0] == len(labels_list_Wav2vec) == len(filepaths_list_Wav2vec), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Wav2vec)}")


--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---
Shape đặc trưng: (5184, 1024), Nhãn: 5184, Filepaths: 5184
Các lớp tìm thấy trong nhãn: ['Anger' 'Happiness' 'Neutral state' 'Sadness']


In [20]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Wav2vec, labels_list_Wav2vec, filepaths_list_Wav2vec, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")


Bước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.
Đã chia dữ liệu:
  Train: X_train shape (4147, 1024), y_train len 4147, train_filepaths len 4147
  Test: X_test shape (1037, 1024), y_test len 1037, test_filepaths len 1037


In [21]:
svc_pipeline = create_specific_svc_pipeline(**SVC_PARAMS)
# Tạo đối tượng SingleModel cho SVC
model_Wav2vec_b_svc = SingleModel(
    name="Wav2vec SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wav2vec_b_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wav2vec_b_svc.evaluate()
if model_Wav2vec_b_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wav2vec_b_svc.get_name()] = model_Wav2vec_b_svc.get_predictions()

Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': False, 'class_weight': None}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Wav2vec SVM

--- Bắt đầu pipeline cho mô hình: Wav2vec SVM ---
Đang huấn luyện mô hình: Wav2vec SVM...
Hoàn thành huấn luyện mô hình: Wav2vec SVM.
Đang dự đoán với mô hình: Wav2vec SVM...
Hoàn thành dự đoán với mô hình: Wav2vec SVM.
Bỏ qua lấy xác suất và tính độ bất định cho 'Wav2vec SVM' vì mô hình không có predict_proba.
--- Hoàn thành pipeline cho mô hình: Wav2vec SVM ---

--- Báo cáo Đánh giá cho Mô hình: Wav2vec SVM ---
               precision    recall  f1-score   support

        Anger       0.67      0.75      0.71       366
    Happiness       0.00      0.00      0.00       122
      Sadness       0.53      0.74      0.62       348
Neutral state       0.70      0.52      0.59       201

     accuracy                           0.61      1037
    macro avg       0.48      0.50 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

In [22]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Wav2vec_svc = SingleModel(
    name="Wav2vec SVM For Ensemble",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wav2vec_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wav2vec_svc.evaluate()
list_of_aud_hub_wav.append(model_Wav2vec_svc)
list_of_aud_hub.append(model_Wav2vec_svc)
if model_Wav2vec_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wav2vec_svc.get_name()] = model_Wav2vec_svc.get_predictions()


Bước 4:## Chuẩn bị Mô hình SVC ##
Đang tạo pipeline cho mô hình SVC...
  Tham số SVC cuối cùng: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale', 'probability': True, 'class_weight': 'balanced'}
Pipeline SVC đã được tạo.
Đã khởi tạo SingleModel: Wav2vec SVM For Ensemble

--- Bắt đầu pipeline cho mô hình: Wav2vec SVM For Ensemble ---
Đang huấn luyện mô hình: Wav2vec SVM For Ensemble...
Hoàn thành huấn luyện mô hình: Wav2vec SVM For Ensemble.
Đang dự đoán với mô hình: Wav2vec SVM For Ensemble...
Hoàn thành dự đoán với mô hình: Wav2vec SVM For Ensemble.
Đang lấy xác suất dự đoán với mô hình: Wav2vec SVM For Ensemble...
Hoàn thành lấy xác suất dự đoán với mô hình: Wav2vec SVM For Ensemble.
Đã tính toán độ bất định cho mô hình: Wav2vec SVM For Ensemble.
--- Hoàn thành pipeline cho mô hình: Wav2vec SVM For Ensemble ---

--- Báo cáo Đánh giá cho Mô hình: Wav2vec SVM For Ensemble ---
               precision    recall  f1-score   support

        Anger       0.64      0.72      0.68       366
   

In [23]:
joblib.dump(model_Wav2vec_svc,"model_Wav2vec_svc.pkl")

['model_Wav2vec_svc.pkl']

### Ensemble

#### Audmodel + Hubert

In [24]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble audmodel + hubert ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_aud_hub,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)


--- Thực hiện và đánh giá Ensemble audmodel + hubert ---
Đã khởi tạo EnsembleModel với 2 mô hình cơ sở.
  Có 2 mô hình cung cấp dữ liệu xác suất.
  Có 2 mô hình cung cấp dữ liệu độ bất định.


In [25]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_aud_hub"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")


# Đánh giá chiến lược Uncertainty Lowest (UL)
Đang thực hiện chiến lược Ensemble: Uncertainty Lowest (UL)

--- Báo cáo Đánh giá cho Uncertainty Lowest (UL) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.69      0.77      0.73       366
    Happiness       0.39      0.17      0.24       122
      Sadness       0.63      0.58      0.60       348
Neutral state       0.63      0.81      0.71       201

     accuracy                           0.64      1037
    macro avg       0.59      0.58      0.57      1037
 weighted avg       0.62      0.64      0.63      1037

Balanced Accuracy (Unweighted): 0.5816
Accuracy (Weighted): 0.6422

Ma trận nhầm lẫn:
[[281   5  20  60]
 [ 33  21  28  40]
 [ 14   6 162  19]
 [ 78  22  46 202]]
----------------------------------------------------


In [26]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_aud_hub_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")


# Đánh giá chiến lược Uncertainty Threshold (UT)
Đang thực hiện chiến lược Ensemble: Uncertainty Threshold (UT) với ngưỡng = 0.9

--- Báo cáo Đánh giá cho Uncertainty Threshold (UT, threshold=0.9) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.70      0.78      0.74       366
    Happiness       0.67      0.08      0.15       122
      Sadness       0.29      0.17      0.21       348
Neutral state       0.18      0.36      0.24       201

     accuracy                           0.41      1037
    macro avg       0.46      0.35      0.33      1037
 weighted avg       0.46      0.41      0.40      1037

Balanced Accuracy (Unweighted): 0.3490
Accuracy (Weighted): 0.4127

Ma trận nhầm lẫn:
[[287   0  62  17]
 [ 32  10  61  19]
 [ 18   2  73 108]
 [ 73   3 214  58]]
----------------------------------------------------


In [27]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_aud_hub"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")


# Đánh giá chiến lược Uncertainty Weighted (UW)
Đang thực hiện chiến lược Ensemble: Uncertainty Weighted (UW)

--- Báo cáo Đánh giá cho Uncertainty Weighted (UW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.70      0.78      0.73       366
    Happiness       0.69      0.09      0.16       122
      Sadness       0.15      0.10      0.12       348
Neutral state       0.07      0.12      0.09       201

     accuracy                           0.34      1037
    macro avg       0.40      0.27      0.27      1037
 weighted avg       0.39      0.34      0.33      1037

Balanced Accuracy (Unweighted): 0.2727
Accuracy (Weighted): 0.3423

Ma trận nhầm lẫn:
[[285   0  66  15]
 [ 32  11  53  26]
 [ 18   2  25 156]
 [ 75   3 236  34]]
----------------------------------------------------


In [28]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_aud_hub"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")


# Đánh giá chiến lược Confidence Weighted (CW)
Đang thực hiện chiến lược Ensemble: Confidence Weighted (CW)

--- Báo cáo Đánh giá cho Confidence Weighted (CW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.39      0.99      0.56       366
    Happiness       0.00      0.00      0.00       122
      Sadness       0.05      0.01      0.01       348
Neutral state       0.02      0.00      0.01       201

     accuracy                           0.35      1037
    macro avg       0.11      0.25      0.15      1037
 weighted avg       0.16      0.35      0.20      1037

Balanced Accuracy (Unweighted): 0.2500
Accuracy (Weighted): 0.3520

Ma trận nhầm lẫn:
[[361   0   5   0]
 [111   0   3   8]
 [145   0   1  55]
 [307   0  38   3]]
----------------------------------------------------


In [29]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_aud_hub"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")


# Đánh giá chiến lược Mean Probability Voting
Đang thực hiện chiến lược Ensemble: Mean Probability Voting

--- Báo cáo Đánh giá cho Mean Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.70      0.78      0.74       366
    Happiness       0.67      0.08      0.15       122
      Sadness       0.14      0.09      0.11       348
Neutral state       0.07      0.13      0.09       201

     accuracy                           0.34      1037
    macro avg       0.40      0.27      0.27      1037
 weighted avg       0.39      0.34      0.33      1037

Balanced Accuracy (Unweighted): 0.2738
Accuracy (Weighted): 0.3443

Ma trận nhầm lẫn:
[[287   0  64  15]
 [ 32  10  54  26]
 [ 18   2  27 154]
 [ 73   3 239  33]]
----------------------------------------------------


In [30]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_aud_hub"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")


# Đánh giá chiến lược Max Probability Voting
Đang thực hiện chiến lược Ensemble: Max Probability Voting

--- Báo cáo Đánh giá cho Max Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.70      0.78      0.74       366
    Happiness       0.53      0.07      0.13       122
      Sadness       0.15      0.10      0.12       348
Neutral state       0.07      0.13      0.09       201

     accuracy                           0.34      1037
    macro avg       0.36      0.27      0.27      1037
 weighted avg       0.37      0.34      0.33      1037

Balanced Accuracy (Unweighted): 0.2720
Accuracy (Weighted): 0.3443

Ma trận nhầm lẫn:
[[287   0  61  18]
 [ 34   9  53  26]
 [ 15   5  26 155]
 [ 74   3 236  35]]
----------------------------------------------------


#### Hubert + Wavlm

In [31]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble hubert + wavlm ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_hub_wav,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)


--- Thực hiện và đánh giá Ensemble hubert + wavlm ---
Đã khởi tạo EnsembleModel với 2 mô hình cơ sở.
  Có 2 mô hình cung cấp dữ liệu xác suất.
  Có 2 mô hình cung cấp dữ liệu độ bất định.


In [32]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_hub_wav"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")


# Đánh giá chiến lược Uncertainty Lowest (UL)
Đang thực hiện chiến lược Ensemble: Uncertainty Lowest (UL)

--- Báo cáo Đánh giá cho Uncertainty Lowest (UL) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.76      0.79      0.78       366
    Happiness       0.51      0.32      0.39       122
      Sadness       0.68      0.65      0.66       348
Neutral state       0.65      0.80      0.72       201

     accuracy                           0.69      1037
    macro avg       0.65      0.64      0.64      1037
 weighted avg       0.68      0.69      0.68      1037

Balanced Accuracy (Unweighted): 0.6406
Accuracy (Weighted): 0.6905

Ma trận nhầm lẫn:
[[289   5  20  52]
 [ 17  39  26  40]
 [ 13  10 161  17]
 [ 60  22  39 227]]
----------------------------------------------------


In [33]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_hub_wav_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")


# Đánh giá chiến lược Uncertainty Threshold (UT)
Đang thực hiện chiến lược Ensemble: Uncertainty Threshold (UT) với ngưỡng = 0.9

--- Báo cáo Đánh giá cho Uncertainty Threshold (UT, threshold=0.9) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.75      0.80      0.77       366
    Happiness       0.62      0.21      0.32       122
      Sadness       0.45      0.29      0.35       348
Neutral state       0.27      0.52      0.36       201

     accuracy                           0.50      1037
    macro avg       0.52      0.46      0.45      1037
 weighted avg       0.54      0.50      0.50      1037

Balanced Accuracy (Unweighted): 0.4552
Accuracy (Weighted): 0.5043

Ma trận nhầm lẫn:
[[292   3  51  20]
 [ 20  26  53  23]
 [ 14   5 105  77]
 [ 65   8 175 100]]
----------------------------------------------------


In [34]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_hub_wav"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")


# Đánh giá chiến lược Uncertainty Weighted (UW)
Đang thực hiện chiến lược Ensemble: Uncertainty Weighted (UW)

--- Báo cáo Đánh giá cho Uncertainty Weighted (UW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.75      0.80      0.77       366
    Happiness       0.60      0.21      0.32       122
      Sadness       0.13      0.08      0.10       348
Neutral state       0.08      0.14      0.10       201

     accuracy                           0.36      1037
    macro avg       0.39      0.31      0.32      1037
 weighted avg       0.39      0.36      0.36      1037

Balanced Accuracy (Unweighted): 0.3090
Accuracy (Weighted): 0.3616

Ma trận nhầm lẫn:
[[291   3  57  15]
 [ 20  26  49  27]
 [ 14   5  29 153]
 [ 65   9 245  29]]
----------------------------------------------------


In [35]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_hub_wav"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")


# Đánh giá chiến lược Confidence Weighted (CW)
Đang thực hiện chiến lược Ensemble: Confidence Weighted (CW)

--- Báo cáo Đánh giá cho Confidence Weighted (CW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.43      0.98      0.60       366
    Happiness       0.86      0.05      0.09       122
      Sadness       0.05      0.01      0.02       348
Neutral state       0.01      0.00      0.01       201

     accuracy                           0.36      1037
    macro avg       0.34      0.26      0.18      1037
 weighted avg       0.27      0.36      0.23      1037

Balanced Accuracy (Unweighted): 0.2610
Accuracy (Weighted): 0.3558

Ma trận nhầm lẫn:
[[357   0   9   0]
 [101   6   6   9]
 [114   0   1  86]
 [250   1  92   5]]
----------------------------------------------------


In [36]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_hub_wav"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")


# Đánh giá chiến lược Mean Probability Voting
Đang thực hiện chiến lược Ensemble: Mean Probability Voting

--- Báo cáo Đánh giá cho Mean Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.75      0.80      0.77       366
    Happiness       0.62      0.21      0.32       122
      Sadness       0.13      0.08      0.10       348
Neutral state       0.08      0.14      0.10       201

     accuracy                           0.36      1037
    macro avg       0.39      0.31      0.32      1037
 weighted avg       0.39      0.36      0.36      1037

Balanced Accuracy (Unweighted): 0.3096
Accuracy (Weighted): 0.3626

Ma trận nhầm lẫn:
[[292   3  56  15]
 [ 20  26  49  27]
 [ 14   5  29 153]
 [ 65   8 246  29]]
----------------------------------------------------


In [37]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_hub_wav"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")


# Đánh giá chiến lược Max Probability Voting
Đang thực hiện chiến lược Ensemble: Max Probability Voting

--- Báo cáo Đánh giá cho Max Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.75      0.80      0.77       366
    Happiness       0.52      0.20      0.29       122
      Sadness       0.14      0.09      0.11       348
Neutral state       0.08      0.15      0.10       201

     accuracy                           0.36      1037
    macro avg       0.37      0.31      0.32      1037
 weighted avg       0.39      0.36      0.36      1037

Balanced Accuracy (Unweighted): 0.3103
Accuracy (Weighted): 0.3645

Ma trận nhầm lẫn:
[[292   3  57  14]
 [ 19  25  51  27]
 [ 12   8  30 151]
 [ 66  12 239  31]]
----------------------------------------------------


#### Audmodel + Hubert + Wavlm

In [38]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble audmodel + hubert + wavlm ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_aud_hub_wav,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)


--- Thực hiện và đánh giá Ensemble audmodel + hubert + wavlm ---
Đã khởi tạo EnsembleModel với 3 mô hình cơ sở.
  Có 3 mô hình cung cấp dữ liệu xác suất.
  Có 3 mô hình cung cấp dữ liệu độ bất định.


In [39]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_aud_hub_wav"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")


# Đánh giá chiến lược Uncertainty Lowest (UL)
Đang thực hiện chiến lược Ensemble: Uncertainty Lowest (UL)

--- Báo cáo Đánh giá cho Uncertainty Lowest (UL) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.73      0.80      0.76       366
    Happiness       0.57      0.30      0.39       122
      Sadness       0.68      0.64      0.66       348
Neutral state       0.64      0.79      0.71       201

     accuracy                           0.68      1037
    macro avg       0.66      0.63      0.63      1037
 weighted avg       0.68      0.68      0.67      1037

Balanced Accuracy (Unweighted): 0.6305
Accuracy (Weighted): 0.6837

Ma trận nhầm lẫn:
[[292   2  22  50]
 [ 25  36  27  34]
 [ 15   7 159  20]
 [ 68  18  40 222]]
----------------------------------------------------


In [40]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_aud_hub_wav_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")


# Đánh giá chiến lược Uncertainty Threshold (UT)
Đang thực hiện chiến lược Ensemble: Uncertainty Threshold (UT) với ngưỡng = 0.9

--- Báo cáo Đánh giá cho Uncertainty Threshold (UT, threshold=0.9) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.73      0.81      0.77       366
    Happiness       0.70      0.17      0.28       122
      Sadness       0.47      0.30      0.37       348
Neutral state       0.27      0.52      0.36       201

     accuracy                           0.51      1037
    macro avg       0.54      0.45      0.44      1037
 weighted avg       0.55      0.51      0.50      1037

Balanced Accuracy (Unweighted): 0.4500
Accuracy (Weighted): 0.5072

Ma trận nhầm lẫn:
[[296   1  49  20]
 [ 25  21  55  21]
 [ 16   2 104  79]
 [ 66   6 171 105]]
----------------------------------------------------


In [41]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_aud_hub_wav"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")


# Đánh giá chiến lược Uncertainty Weighted (UW)
Đang thực hiện chiến lược Ensemble: Uncertainty Weighted (UW)

--- Báo cáo Đánh giá cho Uncertainty Weighted (UW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.74      0.81      0.77       366
    Happiness       0.67      0.16      0.26       122
      Sadness       0.15      0.10      0.12       348
Neutral state       0.07      0.13      0.09       201

     accuracy                           0.36      1037
    macro avg       0.41      0.30      0.31      1037
 weighted avg       0.40      0.36      0.36      1037

Balanced Accuracy (Unweighted): 0.3019
Accuracy (Weighted): 0.3645

Ma trận nhầm lẫn:
[[297   1  52  16]
 [ 25  20  52  25]
 [ 15   2  27 157]
 [ 65   7 242  34]]
----------------------------------------------------


In [42]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_aud_hub_wav"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")


# Đánh giá chiến lược Confidence Weighted (CW)
Đang thực hiện chiến lược Ensemble: Confidence Weighted (CW)

--- Báo cáo Đánh giá cho Confidence Weighted (CW) Ensemble ---
               precision    recall  f1-score   support

        Anger       0.44      0.98      0.60       366
    Happiness       0.86      0.05      0.09       122
      Sadness       0.05      0.01      0.02       348
Neutral state       0.01      0.00      0.01       201

     accuracy                           0.36      1037
    macro avg       0.34      0.26      0.18      1037
 weighted avg       0.27      0.36      0.23      1037

Balanced Accuracy (Unweighted): 0.2610
Accuracy (Weighted): 0.3558

Ma trận nhầm lẫn:
[[357   0   9   0]
 [100   6   6  10]
 [112   0   1  88]
 [250   1  92   5]]
----------------------------------------------------


In [43]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_aud_hub_wav"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")


# Đánh giá chiến lược Mean Probability Voting
Đang thực hiện chiến lược Ensemble: Mean Probability Voting

--- Báo cáo Đánh giá cho Mean Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.73      0.81      0.77       366
    Happiness       0.70      0.17      0.28       122
      Sadness       0.15      0.10      0.12       348
Neutral state       0.07      0.13      0.09       201

     accuracy                           0.36      1037
    macro avg       0.41      0.30      0.31      1037
 weighted avg       0.41      0.36      0.36      1037

Balanced Accuracy (Unweighted): 0.3032
Accuracy (Weighted): 0.3645

Ma trận nhầm lẫn:
[[296   1  54  15]
 [ 25  21  51  25]
 [ 16   2  27 156]
 [ 66   6 242  34]]
----------------------------------------------------


In [44]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_aud_hub_wav"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")


# Đánh giá chiến lược Max Probability Voting
Đang thực hiện chiến lược Ensemble: Max Probability Voting

--- Báo cáo Đánh giá cho Max Probability Voting Ensemble ---
               precision    recall  f1-score   support

        Anger       0.74      0.82      0.78       366
    Happiness       0.54      0.17      0.26       122
      Sadness       0.15      0.10      0.12       348
Neutral state       0.08      0.13      0.10       201

     accuracy                           0.37      1037
    macro avg       0.37      0.31      0.31      1037
 weighted avg       0.39      0.37      0.36      1037

Balanced Accuracy (Unweighted): 0.3060
Accuracy (Weighted): 0.3684

Ma trận nhầm lẫn:
[[300   1  45  20]
 [ 26  21  49  26]
 [ 14   6  27 154]
 [ 68  11 235  34]]
----------------------------------------------------


In [45]:
print("\nToàn bộ quá trình hoàn thành!")


Toàn bộ quá trình hoàn thành!


In [46]:
# 7. Tạo báo cáo misclassification (Sử dụng MisclassificationReporter)
print("\nBước 7: Tạo báo cáo misclassification.")
reporter = MisclassificationReporter(base_output_dir=REPORT_BASE_DIR)

# --- Tạo báo cáo CSV CHỈ CHỨA MẪU SAI cho từng mô hình/chiến lược ---
print("\n--- Tạo báo cáo CSV chỉ chứa mẫu sai ---")
for name, y_pred in ensemble_predictions_dict.items():
    # Lấy nhãn thật (y_test) cho tất cả các dự đoán này
    # Đảm bảo y_pred và y_test có cùng thứ tự mẫu (được đảm bảo bởi workflow)
    reporter.generate_misclassified_csv(
            y_true=y_test, # Nhãn thật của tập test
            y_pred=y_pred, # Dự đoán của mô hình/chiến lược
            sample_identifiers=test_filepaths, # Đường dẫn file của tập test
            report_name=name # Tên báo cáo (tên mô hình/chiến lược)
    )


Bước 7: Tạo báo cáo misclassification.
MisclassificationReporter initialized. Reports will be saved in 'summary_report.csv'.

--- Tạo báo cáo CSV chỉ chứa mẫu sai ---

Đang tạo báo cáo misclassification (chỉ mẫu sai) cho: Wavlm SVM
Đã lưu báo cáo misclassification (chỉ mẫu sai) cho Wavlm SVM vào 'summary_report.csv/Wavlm SVM_misclassified.csv'

Đang tạo báo cáo misclassification (chỉ mẫu sai) cho: Wavlm SVM For Ensemble
Đã lưu báo cáo misclassification (chỉ mẫu sai) cho Wavlm SVM For Ensemble vào 'summary_report.csv/Wavlm SVM For Ensemble_misclassified.csv'

Đang tạo báo cáo misclassification (chỉ mẫu sai) cho: Hubert SVM
Đã lưu báo cáo misclassification (chỉ mẫu sai) cho Hubert SVM vào 'summary_report.csv/Hubert SVM_misclassified.csv'

Đang tạo báo cáo misclassification (chỉ mẫu sai) cho: Hubert SVM For Ensemble
Đã lưu báo cáo misclassification (chỉ mẫu sai) cho Hubert SVM For Ensemble vào 'summary_report.csv/Hubert SVM For Ensemble_misclassified.csv'

Đang tạo báo cáo misclassificat

In [47]:
# --- Tạo báo cáo CSV SO SÁNH chứa TẤT CẢ mẫu test ---
print("\n--- Tạo báo cáo CSV so sánh (tất cả mẫu test) ---")
comparison_report_filename = f"comparison_report_testset_svm.csv" # Tên file dựa trên mô hình nhúng
reporter.generate_comparison_report(
    sample_identifiers=test_filepaths, # Đường dẫn file của TẤT CẢ mẫu test
    y_true=y_test,                   # Nhãn thật của TẤT CẢ mẫu test
    predictions_dict=ensemble_predictions_dict, # Dictionary các dự đoán từ tất cả mô hình/chiến lược
    output_filename=comparison_report_filename
)


--- Tạo báo cáo CSV so sánh (tất cả mẫu test) ---

Đang tạo/cập nhật báo cáo so sánh: comparison_report_testset_svm.csv
Đã lưu báo cáo so sánh vào 'summary_report.csv/comparison_report_testset_svm.csv'


'summary_report.csv/comparison_report_testset_svm.csv'

### Phát triển

#### Thử nghiệm thêm 3 mô hình mới 
<h4> unispeech,nest,canary </h4>

##### Unispeech

In [48]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_unispeech,
#         model_name_or_path=HF_MODEL_NAME_unispeech
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


##### Nest

In [49]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_nest,
#         model_name_or_path=HF_MODEL_NAME_nest
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


##### Canary

In [50]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_canary,
#         model_name_or_path=HF_MODEL_NAME_canary
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")
